In [7]:
import pickle

heterodata = pickle.load(open("hetero_graph_final.pkl", "rb"))

In [2]:
heterodata[('paper', 'in_journal', 'journal')] = heterodata[('paper', 'in_journal', 'journal-ref')]
del heterodata[('paper', 'in_journal', 'journal-ref')]

# remove isolated nodes
import torch_geometric.transforms as T
transform = T.Compose([
       #T.RemoveIsolatedNodes(),
       T.RemoveDuplicatedEdges(),
       # T.ToUndirected(merge=False) # don't merge reversed edges into the original edge type
])

heterodata = transform(heterodata)

heterodata['paper']['name'] = heterodata['paper']['title']
del heterodata['paper']['title']

In [3]:
import torch
# id mapping so we can remove the attributes and then remove isolated nodes, later we can add the attributes back
id_dict = {}
heterodata_dict = heterodata.to_dict()
for nodetype in heterodata.node_types:
    print(nodetype)
    ids = torch.arange(heterodata[nodetype].num_nodes) 
    id_mapping = {i.item():name for i, name in zip(ids, heterodata[nodetype].name)}
    
    heterodata[nodetype].x = ids.unsqueeze(-1)
    id_dict[nodetype] = id_mapping
    
    for key in heterodata_dict[nodetype].keys():
        if key != 'x':
            print('del', nodetype, key)
            del heterodata[nodetype][key]

paper
del paper num_nodes
del paper license
del paper doi
del paper pages
del paper journal
del paper date
del paper id
del paper name
author
del author num_nodes
del author name
category
del category num_nodes
del category name
journal
del journal num_nodes
del journal name
word
del word num_nodes
del word name


In [4]:
# check shapes are correct
for nodetype in heterodata.node_types:
    print(nodetype, heterodata[nodetype].x.shape, heterodata[nodetype].num_nodes)
    print(heterodata[nodetype])

paper torch.Size([2385180, 1]) 2385180
author torch.Size([1878690, 1]) 1878690
category torch.Size([176, 1]) 176
journal torch.Size([2385180, 1]) 2385180
word torch.Size([2406583, 1]) 2406583


In [9]:
# check id mapping is correct
for edge_type in heterodata.edge_types:
    print("     ")
    print(edge_type)
    print(edge_type[0], 'max idx in edge:',heterodata[edge_type].edge_index[0,:].max() ,'actual number of nodes:',heterodata[edge_type[0]].num_nodes)
    print(edge_type[2],'max idx in edge:', heterodata[edge_type].edge_index[1,:].max(),'actual number of nodes:' ,heterodata[edge_type[2]].num_nodes)
    
    

     
('paper', 'written_by', 'author')
paper max idx in edge: tensor(4277325) actual number of nodes: 2385180
author max idx in edge: tensor(4277328) actual number of nodes: 1878690
     
('paper', 'has_category', 'category')
paper max idx in edge: tensor(2381347) actual number of nodes: 2385180
category max idx in edge: tensor(2380598) actual number of nodes: 176
     
('paper', 'has_word', 'word')
paper max idx in edge: tensor(2385179) actual number of nodes: 2385180
word max idx in edge: tensor(725125) actual number of nodes: 2406583
     
('paper', 'has_titleword', 'word')
paper max idx in edge: tensor(2779629) actual number of nodes: 2385180
word max idx in edge: tensor(2779624) actual number of nodes: 2406583
     
('paper', 'in_journal', 'journal-ref')
paper max idx in edge: tensor(1642840) actual number of nodes: 2385180
journal-ref max idx in edge: tensor(1642841) actual number of nodes: None
     
('word', 'co_occurs_with', 'word')
word max idx in edge: tensor(1475095) actua

/home/amosd/miniconda3/envs/gnn/lib/python3.11/site-packages/torch_geometric/data/storage.py:327: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set 'set()'. Please explicitly set 'num_nodes' as an attribute of 'data[journal-ref]' to suppress this warning
  warnings.warn(


In [ ]:
# check correct edge types:
transform = T.Compose([
       #T.RemoveIsolatedNodes(),
       #    T.RemoveDuplicatedEdges(),
       T.ToUndirected(merge=False) # don't merge reversed edges into the original edge type
])

heterodata = transform(heterodata)

In [5]:
transform = T.Compose([
       T.RemoveIsolatedNodes(),
])
heterodata = transform(heterodata)

IndexError: index 3741316 is out of bounds for dimension 0 with size 2385180

In [ ]:
for nodetype in heterodata.node_types:
    print(nodetype, heterodata[nodetype].x.shape, heterodata[nodetype].num_nodes)

In [7]:
def describe(heterodata):
    heterodata_dict = heterodata.to_dict()
    # print('has isolated nodes:', heterodata.has_isolated_nodes())
    # print('has self loops:', heterodata.has_self_loops())
    # print('is directed', heterodata.is_directed())
    for nodetype in heterodata.node_types:
        print(nodetype)
        print('  ',heterodata[nodetype].num_nodes)
        print('  ',heterodata_dict[nodetype].keys())
        
    for edgetype in heterodata.edge_types:
        print(edgetype)
        print('  ',heterodata[edgetype].num_edges)
        print('  ',heterodata[edgetype])

describe(heterodata)

paper
   2385180
   dict_keys(['num_nodes', 'license', 'doi', 'title', 'pages', 'journal', 'date', 'id'])
author
   1878690
   dict_keys(['num_nodes', 'name'])
category
   176
   dict_keys(['num_nodes', 'name'])
journal
   2385180
   dict_keys(['num_nodes', 'name'])
word
   2406583
   dict_keys(['num_nodes', 'name'])
('paper', 'written_by', 'author')
   10597219
   {'edge_index': tensor([[      0,       1,       1,  ..., 4277325, 4277325, 4277325],
        [1589437,   23307, 1051965,  ..., 3198795, 3316631, 4058572]])}
('paper', 'has_category', 'category')
   4037381
   {'edge_index': tensor([[      0,       0,       1,  ..., 2381346, 2381346, 2381347],
        [ 674169, 1788806,  207521,  ..., 1520372, 1862801, 2270900]])}
('paper', 'has_word', 'word')
   138266332
   {'edge_index': tensor([[      0,       0,       0,  ..., 2385179, 2385179, 2385179],
        [  95660,  102536,  115088,  ...,  652992,  684129,  697016]]), 'edge_attr': tensor([0.0693, 0.0758, 0.1286,  ..., 0.1126, 0.03

In [ ]:
heterodata_dict

In [6]:
heterodata_dict['paper'].keys()

dict_keys(['num_nodes', 'license', 'doi', 'title', 'pages', 'journal', 'date', 'id'])

In [ ]:
# check if object has duplicate edges
def get_num_nodes